In [1]:
import sys
import os
import zipfile
import cv2
from PIL import Image
import numpy as np

In [ ]:
class Loader():
    def __init__(self, image_path = None, channels = 3, image_size = 256, batch_size = 1, split_size = 0.20):
        self.image_path = image_path
        self.channels = channels
        self.image_size = image_size
        self.batch_size = batch_size
        self.split_size = split_size
        
    def transforms(self):
        pass
    
    def unzip_folder(self):
        pass
    
    def feature_extractor(self):
        pass
    
    def create_dataloader(self):
        pass